# Rap Music Analysis

In [7]:
import urllib.request
import urllib.parse
import urllib.error
from urllib.request import Request, urlopen
import requests

from bs4 import BeautifulSoup

import pandas as pd
import json

In [8]:
# Create an empty DataFrame
column_names = ['Title','Artist','Lyrics','Comments']
df = pd.DataFrame(columns = column_names)

In [9]:
def request_artist_info(artist_name, page):
    
    base_url = 'https://api.genius.com'
    token = 'VetJWKIjYbo8ujjg8V_PTQnqAkY8xN1fbajgaaTiON0rkgWFuI6X47dXPKCaZQLM'
    
    headers = {'Authorization': 'Bearer ' + token}
    search_url = base_url + '/search?per_page=50&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    
    return response      

In [10]:
response = request_artist_info('Drake', 1)
json = response.json()

In [11]:
json = response.json()

artist = []
song_name = []

for hit in json['response']['hits']: 
    artist.append(hit['result']['primary_artist']['name'])
    song_name.append(hit['result']['title'])

In [14]:
df['Title']=song_name
df['Artist'] = artist

In [15]:
df

,Title,Artist,Lyrics,Comments
0,God’s Plan,Drake,NaN,NaN
1,In My Feelings,Drake,NaN,NaN
2,Hotline Bling,Drake,NaN,NaN
3,One Dance,Drake,NaN,NaN
4,"Hold On, We’re Going Home",Drake,NaN,NaN
5,Know Yourself,Drake,NaN,NaN
6,Back to Back,Drake,NaN,NaN
7,All Me,Drake,NaN,NaN
8,Fake Love,Drake,NaN,NaN
9,From Time,Drake,NaN,NaN


In [17]:
def request_song_url(artist_name, song_cap):
    
    """ This function populates a list with urls to the songs by a chosen artist. 

    Args: 
        artist_name(str): name of the artist
        song_cap(int): number of urls/songs you want 
        
    Returns: 
        list of strings, each string is a url to a song by the artist
    
    """
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()       
        song_info = []
        
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
        
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
    
    return songs

In [22]:
urls = request_song_url('drake',20)

In [25]:
for url in urls: 
    
    req = Request(url, headers = {"User-Agent" : "Mozilla/5.0"})
    webpage = urlopen(req).read()

    # Create a BeautifulSoup object 
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    song={}
    
    song['Lyrics'] = [];
    song['Comments']=[];

    # Extract title of the song
    for title in soup.findAll('title'):    
        song['Title'].append(title.text.strip())

    # Extract user comments on the song
    for div in soup.findAll('div',attrs={'class':'rich_text_formatting'}):
        comments = div.text.strip().split("\n")

        for comment in comments: 
            if comment != "":
                song['Comments'].append(comment);

    # Extract the lyrics of the song
    for div in soup.findAll('div', attrs = {'class':'lyrics'}):
        song['Lyrics'].append(div.text.strip().split("\n"));
   
    df_length = len(df)
    df.loc[df_length] = [song['Lyrics'],song['Comments']]

KeyError: 'Title'

In [ ]:
df.head()

In [ ]:
first_hit

In [ ]:
df['Artist'] = df['Artist'].replace('[]','')
df['Title'] = df['Title'].replace('\xa0',' ')
df['Title'] = df['Title'].str.split('|')[0]

df['Artist'] = df['Title'].str.split('–')[0]
df['Title'] = df['Title'].str.split('–')[1]

In [ ]:
 for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    

In [ ]:
import lyricsgenius as genius
GENIUS_API_TOKEN = 'VetJWKIjYbo8ujjg8V_PTQnqAkY8xN1fbajgaaTiON0rkgWFuI6X47dXPKCaZQLM'
artist_name = 'Drake'

In [ ]:
api=genius.Genius(GENIUS_API_TOKEN)
artist = api.search_artist(artist_name,max_songs=5)

In [ ]:
artist = api.search_artist(artist_name)

The OS module in python provides functions for interacting with the operating system

In [ ]:
import os

os.getcwd()

In [ ]:
artist.save_lyrics()

In [ ]:
import pandas as pd
Artist=pd.read_json("Lyrics_Drake.json")